In [2]:
dataset_path = '../bin/multirc_dataset.hf'
model_name = '/tiedaar/summary-longformer-wording'

In [3]:
from transformers import pipeline
from datasets import DatasetDict
import pandas as pd
from tqdm import tqdm
import torch

tqdm.pandas()

In [4]:
pipe = pipeline('text-classification', model='tiedaar/short-answer-classification', device=0)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
ds = DatasetDict.load_from_disk(dataset_path)

In [6]:
test_df = ds['test'].to_pandas()

In [7]:
df = pd.read_csv('../data/multirc.csv')
column_index = list(df.columns)
column_index[0] = 'index'
df.columns = column_index
test_df = df.merge(test_df, on='index', how='right')
test_df['labels'] = test_df['labels'].progress_apply(lambda x: 'correct_answer' if x==1 else 'incorrect_answer')

In [8]:
test_df['preds'] = test_df['text'].progress_apply(lambda x: pipe(x)[0]['label'])

  0%|          | 7/3962 [00:01<12:01,  5.48it/s]/home/jovyan/conda_envs/wes_env/lib/python3.11/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 3962/3962 [01:00<00:00, 65.88it/s]


In [14]:
test_df[test_df['labels']!=test_df['preds']].to_csv('../results/mpnet-misclassifications.csv')